In [9]:
import numpy as np
from scipy.stats import norm

In [ ]:
#Delta (Δ): Sensitivity to the underlying asset's price.
#Gamma (Γ): Sensitivity of Delta to the underlying asset's price.
#Vega (ν): Sensitivity to volatility.
#Theta (Θ): Sensitivity to the passage of time.
#Rho (ρ): Sensitivity to the interest rate.
#Lambda (λ): Sensitivity to the price of the underlying asset.


In [10]:
def black_scholes(S,K,T,r,sigma, option_type='call'):
    """
    Calculate Black-Scholes option price for European options.

    Parameters:
    - S: Current stock price
    - K: Strike price
    - T: Time to expiration (in years)
    - r: Risk-free interest rate (annual)
    - sigma: Volatility of the underlying stock (Annual)
    - option_type: 'call' or 'put'

    Returns:
    - Option price
    """

    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    if option_type == 'call':
        price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Option type must be either 'call' or 'put'")
    
    return price


In [12]:
def delta(S,K,T,r,sigma, option_type='call'):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))

    if option_type == 'call':
        delta = norm.cdf(d1)
    elif option_type == 'put':
        delta = norm.cdf(d1) - 1
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")
    
    return delta


In [13]:
def gamma(S,K,T,r,sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    gamma_value = norm.pdf(d1)/S*sigma*np.sqrt(T)
    return gamma_value

In [14]:
def vega (S,K,T,r,sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    vega_value = S * norm.pdf(d1) * np.sqrt(T) / 100  # Divided by 100 to express in percentage terms
    return vega_value

In [15]:
def theta(S,K,T,r,sigma, option_type='call'):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    if option_type == 'call':
        theta_value = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) -
                       r * K * np.exp(-r * T) * norm.cdf(d2)) / 365
    elif option_type == 'put':
        theta_value = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) +
                       r * K * np.exp(-r * T) * norm.cdf(-d2)) / 365
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

    return theta_value


In [16]:
def rho(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        rho_value = K * T * np.exp(-r * T) * norm.cdf(d2) / 100
    elif option_type == 'put':
        rho_value = -K * T * np.exp(-r * T) * norm.cdf(-d2) / 100
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

    return rho_value


In [17]:
def calculate_greeks(S, K, T, r, sigma, option_type='call'):
    price = black_scholes(S, K, T, r, sigma, option_type)
    delta_value = delta(S, K, T, r, sigma, option_type)
    gamma_value = gamma(S, K, T, r, sigma)
    vega_value = vega(S, K, T, r, sigma)
    theta_value = theta(S, K, T, r, sigma, option_type)
    rho_value = rho(S, K, T, r, sigma, option_type)

    greeks = {
        'Price': price,
        'Delta': delta_value,
        'Gamma': gamma_value,
        'Vega': vega_value,
        'Theta': theta_value,
        'Rho': rho_value
    }

    return greeks


In [24]:
# Define parameters
S = 100       # Current stock price ($)
K = 100       # Strike price ($)
T = 0.5       # Time to expiration (years)
r = 0.02      # Risk-free interest rate (annualized)
sigma = 0.25  # Volatility (annualized)
option_type = 'call'  # 'call' or 'put'

# Calculate Greeks
greeks = calculate_greeks(S, K, T, r, sigma, option_type)

# Display the results
print(f"Option Type: {option_type.capitalize()}")
print(f"Option Price: ${greeks['Price']:.2f}")
print(f"Delta: {greeks['Delta']:.4f}")
print(f"Gamma: {greeks['Gamma']:.6f}")
print(f"Vega: {greeks['Vega']:.4f}")
print(f"Theta: {greeks['Theta']:.4f}")
print(f"Rho: {greeks['Rho']:.4f}")


Option Type: Call
Option Price: $7.52
Delta: 0.5576
Gamma: 0.000698
Vega: 0.2791
Theta: -0.0218
Rho: 0.2412


In [25]:
import yfinance as yf
import numpy as np

def get_historical_volatility(ticker,period='1y'):
    """
    Calulate the annualized historical volatility of a stock.

    parameters:
    -ticker: Stock ticker symbol
    -period: Period to calculate volatility over. (String '1y','6mo','2y').

    Returns:
    - volatility: Annualized volatility (float) 
    """

    # Get historical data
    stock = yf.Ticker(ticker)
    hist = stock.history(period=period)

    # check if data is available:
    if hist.empty:
        raise ValueError("No data available for the ticker {ticker}.")
    
    # Calculate daily log returns
    hist['Log_Returns'] = np.log(hist['Close']/hist['Close'].shift(1))
    hist.dropna(inplace=True)

    # Calculate the standard deviation of daily log returns
    daily_volatility = hist['Log_Returns'].std()

    # Annualize the volatility
    trading_days = 252
    annualized_volatility = daily_volatility * np.sqrt(trading_days)
    
    return annualized_volatility

In [26]:
def get_risk_free_rate():
    """
    Get the current risk-free rate from the US Treasury yield curve.

    Returns:
    - risk_free_rate: Current risk-free rate (float)
    """

    # US 10-year Treasury yield ticker
    treasury_ticker = '^TNX'

    #fetch latest data
    treasury = yf.Ticker(treasury_ticker)
    hist = treasury.history(period='1d')

    # check if data is available:
    if hist.empty:
        raise ValueError("Failed to fetch risk free rate data.")
    
    #get the last closing price (yield)
    latest_yield = hist['Close'].iloc[-1]

    # Convert yield to risk-free rate
    risk_free_rate = latest_yield / 100

    return risk_free_rate

In [29]:
import numpy as np
from scipy.stats import norm
import yfinance as yf

# Define the Black-Scholes functions (as previously implemented)

def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

    return price

# Greeks functions (delta, gamma, vega, theta, rho) as previously implemented

# ... (Include the delta, gamma, vega, theta, rho functions here)

def calculate_greeks(S, K, T, r, sigma, option_type='call'):
    price = black_scholes(S, K, T, r, sigma, option_type)
    delta_value = delta(S, K, T, r, sigma, option_type)
    gamma_value = gamma(S, K, T, r, sigma)
    vega_value = vega(S, K, T, r, sigma)
    theta_value = theta(S, K, T, r, sigma, option_type)
    rho_value = rho(S, K, T, r, sigma, option_type)

    greeks = {
        'Price': price,
        'Delta': delta_value,
        'Gamma': gamma_value,
        'Vega': vega_value,
        'Theta': theta_value,
        'Rho': rho_value
    }

    return greeks



def main():
    print("Black-Scholes Option Greeks Calculator")
    
    # Get user inputs
    option_type = input("Option Type ('call' or 'put'): ").strip().lower()
    ticker = input("Stock Ticker Symbol (e.g., 'AAPL'): ").strip().upper()
    S = float(input("Current Stock Price (leave blank to fetch automatically): ") or 0)
    K = float(input("Strike Price (K): "))
    T_days = int(input("Time to Expiration in Days (e.g., 90): "))
    T = T_days / 365  # Convert days to years

    # Fetch current stock price if not provided
    if S == 0:
        stock = yf.Ticker(ticker)
        S = stock.history(period='1d')['Close'].iloc[-1]
        print(f"Current Stock Price (S): {S:.2f}")

    # Get volatility
    use_auto_vol = input("Calculate volatility from historical data? (y/n): ").strip().lower()
    if use_auto_vol == 'y':
        period = input("Enter period for volatility calculation (e.g., '6mo', '1y', '2y'): ").strip()
        sigma = get_historical_volatility(ticker, period=period)
        print(f"Calculated Volatility (sigma): {sigma:.4f}")
    else:
        sigma = float(input("Volatility (sigma) in decimal (e.g., 0.25 for 25%): "))

    # Get risk-free interest rate
    use_auto_rf = input("Fetch current risk-free interest rate automatically? (y/n): ").strip().lower()
    if use_auto_rf == 'y':
        r = get_risk_free_rate()
        print(f"Current Risk-Free Interest Rate (r): {r:.4f}")
    else:
        r = float(input("Risk-Free Interest Rate (r) in decimal (e.g., 0.02 for 2%): "))

    # Calculate Greeks
    greeks = calculate_greeks(S, K, T, r, sigma, option_type)

    # Display the results
    print(f"\nOption Type: {option_type.capitalize()}")
    print(f"Underlying Stock: {ticker}")
    print(f"Time to Expiration: {T_days} days ({T:.2f} years)")
    print(f"Option Price: ${greeks['Price']:.2f}")
    print(f"Delta: {greeks['Delta']:.4f}")
    print(f"Gamma: {greeks['Gamma']:.6f}")
    print(f"Vega: {greeks['Vega']:.4f}")
    print(f"Theta: {greeks['Theta']:.4f}")
    print(f"Rho: {greeks['Rho']:.4f}")

if __name__ == "__main__":
    main()


Black-Scholes Option Greeks Calculator
Calculated Volatility (sigma): 0.2237
Current Risk-Free Interest Rate (r): 0.0378

Option Type: Call
Underlying Stock: AAPL
Time to Expiration: 200 days (0.55 years)
Option Price: $1.40
Delta: 0.0853
Gamma: 0.000112
Vega: 0.2667
Theta: -0.0168
Rho: 0.1002
